In [158]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import os
import time
from bs4 import BeautifulSoup
import re
import pandas as pd
import requests


In [159]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# Optional: Configure Chrome options
options = Options()
# Example: options.add_argument("--headless")  # Uncomment if you want to run in headless mode

# Set up the ChromeDriver service
service = Service(ChromeDriverManager().install())

# Create a new instance of the Chrome driver
browser = webdriver.Chrome(service=service, options=options)

# Now you can use `browser` to navigate to pages, etc.


In [162]:
city = "toronto"
product = "Iphone 13"
min_price = 300
max_price = 600
days_listed = 1

In [163]:
url = f'https://www.facebook.com/marketplace/{city}/search/?query={product}&minPrice={min_price}&maxPrice={max_price}&daysSinceListed={days_listed}&exact=false'

browser.get(url)


In [164]:
#decline cookies
try:
    decline_button = browser.find_element(By.XPATH, '//div[@aria-label="Decline optional cookies" and @role="button"]')
    decline_button.click()
    print("Decline optional cookies button clicked!")
except Exception as e:  # It's good practice to catch the exception
    print("Decline optional cookies button not found or could not be clicked.")
    pass


Decline optional cookies button clicked!


In [165]:
#decline cookies
try:
    close_button = browser.find_element(By.XPATH, '//div[@aria-label="Close" and @role="button"]')
    close_button.click()
    print("closed signup")
except Exception as e:  # It's good practice to catch the exception
    print("not found or could not be clicked - signup.")
    pass


closed signup


In [166]:
try:
    close_button = browser.find_element(By.XPATH, '//div[@aria-label="Close" and @role="button"]')
    close_button.click()
    print("Closed signup popup.")
except Exception as e:  # It's good practice to catch the exception
    print("Close signup button not found or could not be clicked.")
    pass


Close signup button not found or could not be clicked.


In [167]:
try:
    close_button = browser.find_element(By.XPATH, '//div[@aria-label="Close" and @role="button"]')
    close_button.click()
    print("Closed signup popup.")
except Exception as e:  # It's good practice to catch the exception
    print("Close signup button not found or could not be clicked.")
    pass

Close signup button not found or could not be clicked.


In [168]:
try:
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        # Scroll to the bottom of the page
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # 4-second pause to allow page to load
        time.sleep(4)
        new_height = browser.execute_script("return document.body.scrollHeight")
        # Check if we've reached the bottom of the page
        if new_height == last_height:
            break

        # Update scroll position for the next iteration
        last_height = new_height

        print("Scrolled")
except Exception as e:
    print(f"Oh no, an error occurred: {e}")

        
        

        




Scrolled
Scrolled
Scrolled
Scrolled
Scrolled
Scrolled


In [169]:
from selenium.common.exceptions import WebDriverException
import time

try:
    last_height = browser.execute_script("return document.body.scrollHeight")
    scroll_attempts = 0  # Track how many times no new content was loaded

    while scroll_attempts < 3:  # Retry scrolling 3 times if no new content loads
        # Scroll to the bottom of the page
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to allow new content to load
        time.sleep(4)

        new_height = browser.execute_script("return document.body.scrollHeight")

        # Check if we've reached the bottom of the page or if new content loaded
        if new_height == last_height:
            scroll_attempts += 1  # No new content, increase attempts counter
        else:
            last_height = new_height  # New content loaded, reset attempts
            scroll_attempts = 0

        print("Scrolled")

    print("Reached the bottom or no more new content loaded.")

except WebDriverException as e:
    print(f"Oh no, an error occurred: {e}")


Scrolled
Scrolled
Scrolled
Reached the bottom or no more new content loaded.


In [170]:
#get the html
html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')

browser.close()


In [173]:

item_links = [link for link in links if product.lower() in link.text.strip().lower() or city.lower() in link.text.strip().lower()]
item_data = []
print(f"Filtered item links: {len(item_links)}")
for item_link in item_links:
    url = item_link.get('href') or 'No URL'
    text = '\n'.join(item_link.stripped_strings)
    item_data.append(f"Text: {text}, URL: {url}")
    # print(f"URL: {url}, Text: {text}")  # Print each item link


Filtered item links: 35


In [175]:
import re

extracted_data = []

for item in item_data:
    # Split the item string
    parts = item.split(", URL: ")
    text = parts[0]  # This contains the text
    url = parts[1] if len(parts) > 1 else 'No URL'  # This contains the URL

    lines = text.split('\n')  # Split the text into lines

    numeric_pattern = re.compile(r'\d[\d,.]*')  # Regex for price
    price = None  # Initialize price

    # Extract price
    for line in lines:
        match = numeric_pattern.search(line)
        if match:
            price_str = match.group()
            price = float(price_str.replace(',', '').replace('CA$', '').strip())  # Convert to float
            print(f"Price extracted: {price}")
            break

    if price is None:
        print("Price not found")

    # Title and location
    title = lines[1] if len(lines) > 1 else 'No title'  # Assuming title is the second line
    location = lines[-1] if lines else 'No location'  # Last line as location
    
    # Extract data into the array
    extracted_data.append({
        'title': title,
        'price': price,
        'location': location,
        'url': url
        #re.sub(r'\?.*'
    })

# Print extracted data
for data in extracted_data:
    print(data)


Price extracted: 455.0
Price extracted: 520.0
Price extracted: 450.0
Price extracted: 470.0
Price extracted: 600.0
Price extracted: 375.0
Price extracted: 500.0
Price extracted: 600.0
Price extracted: 550.0
Price extracted: 550.0
Price extracted: 525.0
Price extracted: 575.0
Price extracted: 440.0
Price extracted: 520.0
Price extracted: 539.0
Price extracted: 539.0
Price extracted: 300.0
Price extracted: 539.0
Price extracted: 300.0
Price extracted: 600.0
Price extracted: 550.0
Price extracted: 470.0
Price extracted: 430.0
Price extracted: 380.0
Price extracted: 550.0
Price extracted: 450.0
Price extracted: 580.0
Price extracted: 600.0
Price extracted: 550.0
Price extracted: 450.0
Price extracted: 485.0
Price extracted: 550.0
Price extracted: 550.0
Price extracted: 570.0
Price extracted: 520.0
{'title': 'Excellent condition white Apple IPhone 13 128 GB', 'price': 455.0, 'location': 'Candiac, QC', 'url': '/marketplace/item/1082804089882334/?ref=search&referral_code=null&referral_story_t

In [176]:
items_df = pd.DataFrame(extracted_data)

sorted_df = items_df.sort_values(by ='price')

cheapest_10 = sorted_df.head(10)

In [177]:
cheapest_10

,title,price,location,url
18,iPhone 13 512GB - Unlocked - New Cash 4 Phones...,300.0,"Ottawa, ON",/marketplace/item/1516645163065576/?ref=search...
16,iPhone 13 (white),300.0,"North Bay, ON",/marketplace/item/3834218316834157/?ref=search...
5,iphone 13 mini 128gb,375.0,"Montréal, QC",/marketplace/item/490229090653147/?ref=search&...
23,"Iphone 13 128gb , 82% battery",380.0,"Montréal, QC",/marketplace/item/1565201557748836/?ref=search...
22,Iphone 13 128gb Red,430.0,"Deux-Montagnes, QC",/marketplace/item/921860356452483/?ref=search&...
12,IPHONE 13 NOIR TRÈS BON ÉTAT 128GB,440.0,"Montréal, QC",/marketplace/item/492838133543202/?ref=search&...
2,iPhone 13 128 Gb,450.0,"Montréal, QC",/marketplace/item/1070938851221745/?ref=search...
29,iPhone 13,450.0,"Notre-Dame-du-Bon-Conseil Paroisse, QC",/marketplace/item/1106925117661859/?ref=search...
25,iPhone 13 128gb + quad lock,450.0,"Ste-Anne-de-Sabrevois, QC",/marketplace/item/8011690112273881/?ref=search...
0,Excellent condition white Apple IPhone 13 128 GB,455.0,"Candiac, QC",/marketplace/item/1082804089882334/?ref=search...


In [178]:
import requests

# Create the message
message = ""
for index, row in cheapest_10.iterrows():
    message += f"Title: {row['title']}\nPrice: {row['price']}\nURL: {row['url']}\n\n"

# Prepare the payload
payload = {
    "content": message  # Send the message as content
}

# Define the correct Discord webhook/channel URL and headers
discord_url = 'add chat url here'

# Fix the typo in 'Authorization' and add "Bot" before the token
headers = {
    "Authorization": "add authorisation key here"
}

# Send the POST request
response = requests.post(discord_url, payload, headers=headers)

# Check response
if response.status_code == 200:
    print("Message sent successfully")
else:
    print(f"Failed to send message, status code: {response.status_code}")


Message sent successfully
